In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_excel('Data/db_master.xlsx',sheet_name='Dist_Detail_Master')

In [ ]:
colu = ['STATE NAME', 'ASM Name', 'SS Code', 'SS NAME', 'SS TOWN',
       'Dist. Code', 'Dealer Name (As Per GST)','Total Target']

In [ ]:
df = df[colu]

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['STATE NAME'].unique()

In [ ]:
df = df[df['STATE NAME'].isin(['Maharashtra','Gujarat', 'Rajasthan'])]
df = df.apply(lambda col: col.str.strip().replace(r'\s+', ' ', regex=True) if col.dtype == "object" else col)

In [ ]:
df['STATE NAME'] = df['STATE NAME'].str.title()
df['ASM Name'] = df['ASM Name'].str.title()
df['SS NAME'] = df['SS NAME'].str.title()
df['Dealer Name (As Per GST)'] = df['Dealer Name (As Per GST)'].str.title()

In [ ]:
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Total Target'] = df['Total Target'].fillna(0)
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Dist. Code'] = df['Dist. Code'].fillna('NA')

In [ ]:
df['SS'] = df['SS NAME']+"|"+df['SS Code']
df['DB'] = df['Dealer Name (As Per GST)']+"|"+df['Dist. Code']

In [ ]:
df.rename(columns={'Total Target':'TGT','STATE NAME':'STATE','ASM Name':'ASM'},inplace=True)

In [ ]:
df.columns

In [ ]:
df.isna().sum()


In [ ]:
col2 = ['STATE','SS','ASM','DB','TGT']

In [ ]:
df1 = df[col2]

In [ ]:
df1['ASM'].unique()

# Read ASM Password

In [ ]:
pas = pd.read_excel('Data/asm.xlsx',usecols=['ASM','Password'])

In [ ]:
df2 = df1[['STATE','SS','ASM']].drop_duplicates()
mer = pd.merge(df2,pas,on='ASM',how='left')
mer['Password'] = mer['Password'].fillna(2025)
df2 = mer

In [ ]:
df3 = df1[['SS','ASM','DB','TGT']]

In [ ]:
df3.sort_values(by=['DB','ASM']).to_json('Output/dbtgt.json')

In [ ]:
df3.columns

# SS to ASM

In [ ]:
df2.columns

In [ ]:
# Create nested dictionary with password at DB level
nested = {}

for _, row in df2.iterrows():
    state = row['STATE']
    ss = row['SS']
    db = row['ASM'] # DB Name
    pwd = row['Password']

    state_dict = nested.setdefault(state, {})
    ss_dict = state_dict.setdefault(ss, {})
    db_dict = ss_dict.setdefault(db, {"password": pwd})


# Save
with open("Output/db.json", "w") as f:
    json.dump(nested, f, indent=4)